In [16]:
#!pip install html5lib bs4 requests lingua-language-detector google-api-python-client google-auth-httplib2 google-auth-oauthlib google-analytics-data selenium --upgrade

#Import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from lingua import Language, LanguageDetectorBuilder
from datetime import date
import calendar
import os.path
import os
import io
import requests
import concurrent.futures
from googleapiclient.http import MediaIoBaseDownload
from google.auth.transport.requests import Request                                             
from google.oauth2.credentials import Credentials                                              
from google_auth_oauthlib.flow import InstalledAppFlow                                       
from googleapiclient.discovery import build                                                  
from googleapiclient.errors import HttpError                                                   
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from difflib import SequenceMatcher

#Set-up English or Indonesian language detector for checking the webscraper
detector = LanguageDetectorBuilder.from_languages(Language.ENGLISH,Language.INDONESIAN).build()

In [17]:
#FUNCTIONS AND OPTIONS

#Sets Selenium Chrome driver to run in headless mode for efficiency
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')

#Function to pull and parse html from a url using Selenium
#Waits until the specified wait tag, h4 header for example, appears in the url
#Overcomes javascripting present in certain websites
def selscraper(url,waittype,waittag):
    chrome_driver.get(url)
    wait = WebDriverWait(chrome_driver,10)
    wait.until(EC.presence_of_element_located((waittype,waittag)))
    html_content = chrome_driver.page_source
    soup = BeautifulSoup(html_content,'html.parser')
    return soup

#Pulls the publication title using the basic Requests library and BeautifulSoup
#Function will run again if an error is raised
def fetch_title(url):
    work = False
    while work == False:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text)
            title = soup.find(id='publication-title')
            if title is None:
                work = True
                return None
            else:
                work = True
                return title.get_text("").replace('\n','').strip()
        except requests.exceptions.RequestException as e:
            work = False

#Returns the string most similar to the input string from a range of strings 
def get_most_similar_string(input_str,string_list):
    similarity_scores = np.array([SequenceMatcher(None,input_str,s).ratio() for s in string_list])
    most_similar_index = np.argmax(similarity_scores)
    return string_list[most_similar_index]

In [18]:
#WEBSCRAPING PUBLICATION TITLES FROM CIPS AND NELITI

#Initialise Chrome Driver for Selenium
chrome_driver = webdriver.Chrome(options=chrome_options)

#CIPS WEBSITE
#Pulls all the English publication titles from the CIPS website using Selenium and saves them to english_titles
#Retries if a title language error is detected, as it has happened in the past
eng_titles_soup = selscraper("https://www.cips-indonesia.org/publications",By.CSS_SELECTOR,'h4')
english_titles = [x.get_text(" ").strip() for x in eng_titles_soup.find_all('h4') if x.get_text(" ") != 'CIPS has more than 90 publications']
while str(detector.detect_language_of(str(english_titles[0]))).replace('Language.','').title() == 'Indonesian':
    eng_titles_soup = selscraper("https://www.cips-indonesia.org/publications",By.CSS_SELECTOR,'h4')
    english_titles = [x.get_text(" ").strip() for x in eng_titles_soup.find_all('h4') if x.get_text(" ") != 'CIPS has more than 90 publications']

#Repeats the same with the Indonesian titles
ind_titles_soup = selscraper("https://www.cips-indonesia.org/publications?lang=id",By.CSS_SELECTOR,'h4')
indonesian_titles = [x.get_text(" ").strip() for x in ind_titles_soup.find_all('h4') if x.get_text(" ") != 'CIPS memiliki lebih dari 90 publikasi']
while str(detector.detect_language_of(str(indonesian_titles[-1]))).replace('Language.','').title() == 'English':
    ind_titles_soup = selscraper("https://www.cips-indonesia.org/publications?lang=id",By.CSS_SELECTOR,'h4')
    indonesian_titles = [x.get_text(" ").strip() for x in ind_titles_soup.find_all('h4') if x.get_text(" ") != 'CIPS memiliki lebih dari 90 publikasi']

#Combines both language title lists into one
all_titles = english_titles + indonesian_titles

#Fixes the titles that do not match those on Google Analytics. Titles need to match for performance matching to upload to the dashboard
for index,x in enumerate(all_titles):
    if x == 'Perjanjian Regional Comprehensive Economic Partnership (RCEP):  Peluangnya bagi Indonesia dan Langkah Pemanfaatannya  Sebuah Perspektif Internal':
        all_titles[index] = 'Perjanjian Regional Comprehensive Economic Partnership (RCEP): Peluangnya bagi Indonesia dan Langkah Pemanfaatannya\u2028 Sebuah Perspektif Internal'
    elif x == 'Menuju Sistem Agropangan yang Lebih Berkelanjutan di Indonesia':
        all_titles[index] = 'Menuju Sistem Pertanian Pangan yang Lebih Berkelanjutan di Indonesia'

#NELITI
#Returns the number of pages filled with titles on the Neliti website
neliti_soup = selscraper("https://repository.cips-indonesia.org/browse/all",By.CLASS_NAME,'sr-title')
no_pages = neliti_soup.find_all(class_='pages')[-1].find('a').get_text()

#Loops through all the pages, pulls the titles, and adds them to nel_titles
nel_titles = []
for i in list(range(1,int(no_pages)+1,1)):
    soup = selscraper("https://repository.cips-indonesia.org/browse/all?page="+str(i),By.CLASS_NAME,'sr-title')
    page_titles = [x.get_text().replace('\n','').strip() for x in soup.find_all(class_='sr-title')]
    nel_titles.append(page_titles)
nel_titles = [i for x in nel_titles for i in x]

#Quits the Chrome Driver
chrome_driver.quit()

In [31]:
#PULLING CURRENT GOOGLE SHEETS DASHBOARD AND NELITI FILES

#Sets the appropriate credentials to use the Google Sheets and Drive APIs
#Opens a tab to sign in with Google using the Google Cloud credentials if no working token exists
#In case of credential errors, delete existing token and credentials and redownload from Google Cloud
def credentials():
    scopes = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]
    credentials = None
    if os.path.exists("API_Files/token.json"):
        credentials = Credentials.from_authorized_user_file("API_Files/token.json",scopes)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("API_Files/credentials.json",scopes)
            credentials = flow.run_local_server(port=0)
        with open("API_Files/token.json","w") as token:
            token.write(credentials.to_json())
    return credentials
credentials = credentials()

#Builds the Google Sheets service and executes a batch get for the relevant sheets
service = build("sheets","v4",credentials=credentials)
ranges = ["Website Downloads","Neliti Downloads","Neliti Views"]
batch_response = service.spreadsheets().values().batchGet(spreadsheetId="SPREADSHEETID",ranges=ranges).execute()

#Converts the the sheets into dataframes and sets the appropriate data types
web_dl = batch_response.get("valueRanges")[0].get("values",[])
web_dl = pd.DataFrame(web_dl[1:],columns=web_dl[0])
web_dl = pd.concat([web_dl.iloc[:,:3],web_dl.iloc[:,3:].astype('int64')],axis=1)
nel_dl = batch_response.get("valueRanges")[1].get("values",[])
nel_dl = pd.DataFrame(nel_dl[1:],columns=nel_dl[0])
nel_dl = pd.concat([nel_dl.iloc[:,:3],nel_dl.iloc[:,3:].astype('int64')],axis=1)
nel_view = batch_response.get("valueRanges")[2].get("values",[])
nel_view = pd.DataFrame(nel_view[1:],columns=nel_view[0])
nel_view = pd.concat([nel_view.iloc[:,:3],nel_view.iloc[:,3:].astype('int64')],axis=1)

#Checks the shapes of Neliti Downloads and Neliti Views, should be the same
if nel_dl.shape != nel_view.shape:
    print("Unify columns for Neliti views and Neliti downloads")

elif len(nel_dl.columns.tolist()) == len(nel_view.columns.tolist()):
    
    #Builds the Google Drive service and saves the file names and ids in the Neliti data folder to a dataframe
    service = build("drive","v3",credentials=credentials)
    neliti_files = pd.DataFrame(service.files().list(q="'FOLDERID' in parents",fields="nextPageToken, files(id, name)")\
                                .execute().get("files",[]))

    #Returns the ids and filenames for months in the Neliti data folder that are not currently in the dashboard
    neliti_missing_ids = [neliti_files.loc[neliti_files['name'] == x,'id'].values[0]\
                          for x in neliti_files['name'].tolist() if x.replace('.xlsx','') not in nel_dl.columns.tolist()[3:]]
    neliti_missing_filenames = [neliti_files.loc[neliti_files['name'] == x,'name'].values[0]\
                                for x in neliti_files['name'].tolist() if x.replace('.xlsx','') not in nel_dl.columns.tolist()[3:]]
    
    #Downloads the missing months' files (if any)
    if len(neliti_missing_ids) != 0:
        for id,name in zip(neliti_missing_ids,neliti_missing_filenames):
            request = service.files().get_media(fileId=id)
            byter = io.BytesIO()
            downloader = MediaIoBaseDownload(fd=byter,request=request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print('Download progress {0}'.format(status.progress() * 100))
            byter.seek(0)
            with open('Data/Neliti_Files/'+name,'wb') as f:
                f.write(byter.read())
                byter.close()

Download progress 100.0


In [20]:
#MATCH GOOGLE ANALYTICS TITLES AND TITLES ON CIPS WEBSITE
#Titles need to match to match the downloads
#Modernizing Indonesia's Agriculture, Memodernisasi Pertanian Indonesia, and Kerahasiaan Data dalam Peraturan Perundang-Undangan Perlindungan Data Pribadi di Indonesia
#The above do not have any downloads which is why they appear here. If other titles appear, check Google Analytics for the appropriate format

#Uses service account credentials to setup the Google Analytics service
credentials = service_account.Credentials.from_service_account_file('API_Files/SERVICEACCOUNT.JSON')
client = BetaAnalyticsDataClient(credentials=credentials)

#Runs a report pulling all titles and corresponding file downloads since January 2018 and saves the titles to a list
report = client.run_report({"property":"properties/ID","date_ranges":[{"start_date":"2018-01-01","end_date":"today"}],\
                            "dimensions":[{"name":"eventName","name":"unifiedScreenClass"}],"metrics":[{"name":"eventCount"}],\
                            "dimension_filter":{"filter":{"field_name":"eventName","string_filter":{"value":"file_download"}}}})
ganalytics_data = [row.dimension_values[0].value for row in report.rows]


#Returns a list of titles from the webscraped titles which are not found on Google Analytics
print("Titles on Website but not found on Google Analytics:")
missing_titles = [print(x) for x in all_titles if x not in ganalytics_data]

Titles on Website but not found on Google Analytics:
Modernizing Indonesia’s Agriculture
Memodernisasi Pertanian Indonesia
Kerahasiaan Data dalam Peraturan Perundang-Undangan Perlindungan Data Pribadi di Indonesia


In [24]:
#ADDING NEW DATA TO WEBSITE DOWNLOADS

#Loops through titles on the website but which have not been added to the dashboard
for title in [x for x in all_titles if x not in web_dl['Title'].tolist()]:
    print(f"Title: '{title}'")

    #Asks the user to input the publication type from a list of 4 options
    possible_types = ['Discussion Paper','Policy Brief','Book','Policy Paper']
    for index,y in enumerate(possible_types):
        print(f'{index+1}. {y}')
    input_type = str(input('Please select a number from the list'))
    while input_type not in list(map(str,range(1,len(possible_types)+1,1))):
        input_type = str(input('Please select a number from the list'))
    input_type = possible_types[int(input_type)-1]

    #Asks the user to input the publication topic from a list of 6 options
    possible_topics = ['Digital & Financial Literacy','Economic Opportunities','Community Livelihoods','Education','Food Security & Agriculture','Trade & Investment']
    for index,z in enumerate(possible_topics):
        print(f'{index+1}. {z}')
    input_topic = str(input('Please select a number from the list'))
    while input_topic not in list(map(str,range(1,len(possible_topics)+1,1))):
        input_topic = str(input('Please select a number from the list'))
    input_topic = possible_topics[int(input_topic)-1]

    #Appends the new publication to the bottom of web_dl with the inputted type and topic
    web_dl.loc[len(web_dl)] = [title,input_type,input_topic] + [0] * len([x for x in web_dl.columns.tolist() if x != 'Title' and x != 'Type' and x != 'Topic'])

#Ensures the correct data types
web_dl = pd.concat([web_dl.iloc[:,:3],web_dl.iloc[:,3:].astype('int64')],axis=1)

#Loops through any months that should be in the dashboard but which are not added yet
for month in [x[:7] for x in pd.date_range('2019-01-01',date.today(),freq='M').astype('str').tolist() if x[:7] not in web_dl.columns.tolist()[3:]]:
    current_year, current_month = map(int,month.split('-'))
    last_day = str(calendar.monthrange(current_year,current_month)[1])
    report = client.run_report({"property":"properties/ID","date_ranges":[{"start_date":month+"-01","end_date":month+"-"+last_day}],\
                            "dimensions":[{"name":"eventName","name":"unifiedScreenClass"}],"metrics":[{"name":"eventCount"}],\
                            "dimension_filter":{"filter":{"field_name":"eventName","string_filter":{"value":"file_download"}}}})
    ganalytics_data = pd.DataFrame([{'Page Title':row.dimension_values[0].value,'Downloads':row.metric_values[0].value} for row in report.rows])
    ganalytics_data['Downloads'] = ganalytics_data['Downloads'].astype('int64')

    #Adds the publication data for the missing months to the dataframe
    web_dl[month] = [ganalytics_data.loc[ganalytics_data['Page Title'] == x,'Downloads'].values[0] if x in ganalytics_data['Page Title'].tolist() else 0 for x in web_dl['Title'].tolist()]

In [38]:
#ADDING NEW DATA TO NELITI DOWNLOADS & VIEWS

#Loops through titles on Neliti which are not in the dashboard
for title in [x for x in nel_titles if x not in nel_dl['Title'].tolist()]:
    print(title)

    #Not all titles should be added as there are some duplicates on the website. When unsure, check the website to see which one has data
    cont = input('Add to dashboard? Y/N')
    while cont.lower() != 'y' and cont.lower() != 'n':
        cont = input('Press Y for yes or N for no')

    #Skip the title if it should not be added
    if cont == 'n':
        continue

    #Asks the user for publication type and topic like for Website Downloads then appends each to the Neliti Downloads and Neliti Views dataframes respectively
    elif cont == 'y':
        possible_types = ['Discussion Paper','Policy Brief','Book','Policy Paper']
        for index,y in enumerate(possible_types):
            print(f'{index+1}. {y}')
        input_type = str(input('Please select a number from the list'))
        while input_type not in list(map(str,range(1,len(possible_types)+1,1))):
            input_type = str(input('Please select a number from the list'))
        input_type = possible_types[int(input_type)-1]
    
        possible_topics = ['Digital & Financial Literacy','Economic Opportunities','Community Livelihoods','Education','Food Security & Agriculture','Trade & Investment']
        for index,z in enumerate(possible_topics):
            print(f'{index+1}. {z}')
        input_topic = str(input('Please select a number from the list'))
        while input_topic not in list(map(str,range(1,len(possible_topics)+1,1))):
            input_topic = str(input('Please select a number from the list'))
        input_topic = possible_topics[int(input_topic)-1]

        nel_dl.loc[len(nel_dl)] = [title,input_type,input_topic] + [0] * len([x for x in nel_dl.columns.tolist() if x != 'Title' and x != 'Type' and x != 'Topic'])
        nel_view.loc[len(nel_view)] = [title,input_type,input_topic] + [0] * len([x for x in nel_view.columns.tolist() if x != 'Title' and x != 'Type' and x != 'Topic'])

#Loops through the downloaded month data. The files are only downloaded if missing from the dashboard
for file in os.listdir("Data/Neliti_Files"):

    #Converts the Neliti data file into a dataframe and performs some adjustments including adding a label column for whether the row is a Download or View
    month = pd.read_excel("Data/Neliti_Files/"+file,sheet_name=0,names=['Pages','Hits'])
    month['Pages'] = month['Pages'].str.replace('center-for-indonesian-policy-studies - ','https://')
    month['Type'] = month['Pages'].apply(lambda x: 'Download' if '.pdf' in x else 'View')
    month.sort_values('Type',ascending=False,inplace=True)

    #Uses multithreading to return the publication title for all View data as each entry is a link to the relevant publication on neliti
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        title_views = list(executor.map(fetch_title,month.loc[month['Type'] == 'View','Pages'].tolist()))

    #For download data, extracts the relevant title section from the url
    title_downloads = month.loc[month['Type'] == 'Download','Pages'].apply(lambda x: x.split('publications/')[1].split('-',1)[1].replace('.pdf','').replace('-',' ')).tolist()

    #Adds the extracted titles to the dataframe
    month['Month Titles'] = title_views + title_downloads

    #Converts the extracted titles to the most similar title found in the Neliti dashboard
    month['Dashboard Titles'] = [None if x is None else get_most_similar_string(x,nel_dl['Title'].tolist()) for x in month['Month Titles'].tolist()]

    #Adds a new column for each missing month with the missing data to the Neliti Downloads and Neliti Views dataframes
    nel_dl[file.replace('.xlsx','')] = [month.loc[(month['Dashboard Titles'] == x) & (month['Type'] == 'Download'),'Hits'].values[0]\
                                        if x in month.loc[month['Type'] == 'Download','Dashboard Titles'].tolist() else 0 for x in nel_dl['Title'].tolist()]
    nel_view[file.replace('.xlsx','')] = [month.loc[(month['Dashboard Titles'] == x) & (month['Type'] == 'View'),'Hits'].values[0]\
                                          if x in month.loc[month['Type'] == 'View','Dashboard Titles'].tolist() else 0 for x in nel_view['Title'].tolist()]

Indonesian Food Trade Policy during COVID-19


Add to dashboard? Y/N n


Effects of High Food Prices on Non-Cash Food Subsidies (BPNT) in Indonesia - Case Study in East Nusa Tenggara


Add to dashboard? Y/N n


Political Economy of Rice Policy in Indonesia: A Perspective on the ASEAN Economic Opportunity


Add to dashboard? Y/N n


242
<class 'list'>
<class 'list'>


In [75]:
#DELETING NELITI FILES AND UPLOADING DATAFRAMES TO DASHBOARD

#Deletes all the downloaded files in the folder
for file in os.listdir("Data/Neliti_Files"):
    os.remove("Data/Neliti_Files/" + file)

#Sets credentials
def credentials():
    scopes = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]
    credentials = None
    if os.path.exists("API_Files/token.json"):
        credentials = Credentials.from_authorized_user_file("API_Files/token.json",scopes)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("API_Files/credentials.json",scopes)
            credentials = flow.run_local_server(port=0)
        with open("API_Files/token.json","w") as token:
            token.write(credentials.to_json())
    return credentials
credentials = credentials()

#Sets the relevant ranges with the updated dataframes
service = build("sheets","v4",credentials=credentials)
body = {'values':[web_dl.columns.tolist()] + web_dl.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEETID",range='Website Downloads',valueInputOption="USER_ENTERED",body=body).execute()
body = {'values':[nel_dl.columns.tolist()] + nel_dl.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEETID",range='Neliti Downloads',valueInputOption="USER_ENTERED",body=body).execute()
body = {'values':[nel_view.columns.tolist()] + nel_view.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEETID",range='Neliti Views',valueInputOption="USER_ENTERED",body=body).execute()